<font size="6"> **Model Building** </font>

In [1]:
%load_ext autoreload
%autoreload 2
%run ../nb_config.py

In [2]:
RND_SEED = 123
MODEL_VERSION = 'v01'
FLG_WRITE_SAMPLES = False

In [3]:
import os

import numpy as np
import scipy
from sklearn.metrics import make_scorer

from src import utils
from src.load_data import io_utils
from src.mle_quant_utils import mle_utils

In [4]:
cfg = utils.read_conf()

INFILE1 = cfg['output']['interim']['all_factors']
INFILE2 = cfg['output']['interim']['all_forwreturns']
INFILE3 = cfg['output']['interim']['sector_factors']

TAU = cfg['models']['alpha_ml']['tau']
target_col = cfg['models']['alpha_ml']['target_col']

splits = cfg['models']['alpha_ml']['splits']
split_col = cfg['models']['alpha_ml']['split_col']

In [5]:
split_col = cfg['models']['alpha_ml']['split_col']

# Load Data

In [6]:
all_factors = pd.read_csv(io_utils.interim_path + INFILE1, parse_dates=['date'], index_col=[0,1])
pd.concat([all_factors.head(2), all_factors.tail(2)], axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-12-28 00:00:00+00:00 ZION                                 0.773366   
                          ZTS                                 -0.944433   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-12-28 00:00:00+00:00 ZION               -0.188031   
                          ZTS                 0.209727   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-12-28 00:00:00+00:00 ZION                       1.325552 -0.965002   
                          ZTS                        0.441851  0.229762   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-12-28 00:00:00+00:00 ZION  -0.724635         0.014916        0.014731   
                          ZTS   -0.180275         0.014916        0.014731   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-12-28 00:00:00+00:00 ZION          0.183680        0.181479   
                          ZTS           0.183680        0.181479   

                                 volatility_120d  volatility_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A            -0.836546       -1.219809   
                          AAL           1.639924        1.566220   
2015-12-28 00:00:00+00:00 ZION          0.547895        1.099324   
                          ZTS           0.413572       -0.858958   

                                 sector_code_-1  sector_code_0  sector_code_1  \
date                      asset                                                 
2013-01-03 00:00:00+00:00 A                   0              1              0   
                          AAL                 0              0              0   
2015-12-28 00:00:00+00:00 ZION                0              0              0   
                          ZTS                 0              1              0   

                                 sector_code_2  sector_code_3  sector_code_4  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              1              0   
2015-12-28 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_5  sector_code_6  sector_code_7  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-12-28 00:00:00+00:00 ZION               1              0              0   
                          ZTS                0              0              0   

                                 sector_code_8

In [7]:
all_frets = pd.read_csv(io_utils.interim_path + INFILE2, parse_dates=['date'], index_col=[0,1])
pd.concat([all_frets.head(2), all_frets.tail(2)], axis=0)

return_5d  return_5d_25p  return_5d_5p  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A              0              2             0   
                          AAL            1             22             4   
2015-12-28 00:00:00+00:00 ZION           1             17             3   
                          ZTS            1             14             2   

                                 return_5d_raw  return_5d_fshift  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A           0.013820         -0.009169   
                          AAL         0.056681          0.107664   
2015-12-28 00:00:00+00:00 ZION        0.040065         -0.033658   
                          ZTS         0.032838         -0.013144   

                                 return_5d_fshift_is_positive  
date                      asset                                
2013-01-03 00:00:00+00:00 A                               0.0  
                          AAL                             1.0  
2015-12-28 00:00:00+00:00 ZION                            0.0  
                          ZTS                             0.0

In [8]:
targets_df = all_frets['return_5d_fshift']
targets_df.name = target_col
features_df = all_factors.drop(columns=split_col)
split = all_factors[split_col]

In [9]:
all_factors = all_factors.join(targets_df)

In [10]:
split.value_counts(normalize=True)

train    0.595147
test     0.203870
valid    0.200983
Name: split, dtype: float64

In [11]:
sector_factor_df =  pd.read_csv(io_utils.interim_path + INFILE3, parse_dates=['date'], index_col=[0,1])

# Sampling

In [12]:
all_factors.shape

(361284, 40)

In [13]:
idx_dt = all_factors.index.get_level_values(0)
sampling = np.random.randint(0, 4+1, len(idx_dt))

all_factors_smp = all_factors.loc[idx_dt.weekday == sampling].copy()

pd.concat([all_factors_smp.head(2), all_factors_smp.tail(2)], axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 AAPL                                -1.128796   
                          ABC                                 -0.070318   
2015-12-28 00:00:00+00:00 XYL                                  0.730599   
                          ZBH                                 -0.245909   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 AAPL                1.148508   
                          ABC                -0.464607   
2015-12-28 00:00:00+00:00 XYL                -0.101247   
                          ZBH                -1.084794   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 AAPL                       0.066193  1.728377   
                          ABC                        0.669287  0.176515   
2015-12-28 00:00:00+00:00 XYL                       -0.406503 -1.537640   
                          ZBH                       -1.290204 -0.159066   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 AAPL   1.728377         0.013270        0.011178   
                          ABC    0.227999         0.013270        0.011178   
2015-12-28 00:00:00+00:00 XYL   -1.622475         0.014916        0.014731   
                          ZBH   -0.314598         0.014916        0.014731   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 AAPL          0.127654        0.135452   
                          ABC           0.127654        0.135452   
2015-12-28 00:00:00+00:00 XYL           0.183680        0.181479   
                          ZBH           0.183680        0.181479   

                                 volatility_120d  volatility_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 AAPL          1.050289        1.617813   
                          ABC          -1.197697       -1.367218   
2015-12-28 00:00:00+00:00 XYL          -0.759983       -0.300458   
                          ZBH          -1.000350       -1.276065   

                                 sector_code_-1  sector_code_0  sector_code_1  \
date                      asset                                                 
2013-01-03 00:00:00+00:00 AAPL                0              0              1   
                          ABC                 0              1              0   
2015-12-28 00:00:00+00:00 XYL                 0              0              0   
                          ZBH                 0              1              0   

                                 sector_code_2  sector_code_3  sector_code_4  \
date                      asset                                                
2013-01-03 00:00:00+00:00 AAPL               0              0              0   
                          ABC                0              0              0   
2015-12-28 00:00:00+00:00 XYL                0              1              0   
                          ZBH                0              0              0   

                                 sector_code_5  sector_code_6  sector_code_7  \
date                      asset                                                
2013-01-03 00:00:00+00:00 AAPL               0              0              0   
                          ABC                0              0              0   
2015-12-28 00:00:00+00:00 XYL                0              0              0   
                          ZBH                0              0              0   

                                 sector_code_8

In [14]:
all_factors_smp.shape

(72574, 40)

In [15]:
features_full = features_df.columns.tolist()

In [16]:
features = ['Mean_Reversion_Sector_Neutral_Smoothed', 'Momentum_1YR_Smoothed', 'Overnight_Sentiment_Smoothed', 
            'adv_20d', # 'adv_120d'
             'volatility_20d',
       'dispersion_120d', 'dispersion_20d', 'market_vol_120d','market_vol_20d',
       'sector_code_-1', 'sector_code_0', 'sector_code_1', 'sector_code_2',
       'sector_code_3', 'sector_code_4', 'sector_code_5', 'sector_code_6',
       'sector_code_7', 'sector_code_8', 'sector_code_9', 'sector_code_10',
        'is_December', 'qtr_end', 'qtr_start']

In [17]:
features_nov = ['Mean_Reversion_Sector_Neutral_Smoothed', 'Momentum_1YR_Smoothed', 'Overnight_Sentiment_Smoothed',
 'adv_120d', 'adv_20d',
 'dispersion_120d', 'dispersion_20d',
 'market_vol_120d', 'market_vol_20d',
 'volatility_120d', 'volatility_20d',
 'is_January', 'is_December',
 'month_start', 'month_end',
 'qtr_end', 'qtr_start',
 'sector_code_-1', 'sector_code_0', 'sector_code_1', 'sector_code_2',
 'sector_code_3', 'sector_code_4', 'sector_code_5', 'sector_code_6',
 'sector_code_7', 'sector_code_8', 'sector_code_9', 'sector_code_10',
 'qtr_1', 'qtr_2', 'qtr_3', 'qtr_4']

# Data Splitting

In [18]:
X_train_full = all_factors.loc[split == 'train', features_full].copy()
y_train_full = all_factors.loc[split == 'train', target_col].copy()

X_valid_full = all_factors.loc[split == 'valid', features_full].copy()
y_valid_full = all_factors.loc[split == 'valid', target_col].copy()


X_test_full = all_factors.loc[split == 'test', features_full].copy()
y_test_full = all_factors.loc[split == 'test', target_col].copy()

In [19]:
train_dates = split[split=='train'].index.get_level_values(0)
test_dates = split[split=='test'].index.get_level_values(0)

# Scoring Function

In [20]:
direction_accuracy = make_scorer(mle_utils.direction_accuracy_func, greater_is_better=True, kind='global')
upwards_direction_accuracy = make_scorer(mle_utils.direction_accuracy_func, greater_is_better=True, kind='upwards')
downwards_direction_accuracy = make_scorer(mle_utils.direction_accuracy_func, greater_is_better=True, kind='downwards')
weighted_direction_accuracy = make_scorer(mle_utils.direction_accuracy_func, greater_is_better=True, kind='weighted', w_fp=2., w_fn=1.)

In [21]:
scoring = {'neg_root_mean_squared_error':'neg_root_mean_squared_error',
           'w_accuracy': weighted_direction_accuracy,
           'accuracy': direction_accuracy,
          'up_accuracy': upwards_direction_accuracy,
          'down_accuracy': downwards_direction_accuracy}

# (Optional) Write Train Sample

In [22]:
if FLG_WRITE_SAMPLES:
    X_train_smp = all_factors_smp.loc[split == 'train', features_full]
    y_train_smp = all_factors_smp.loc[split == 'train', [target_col]]
    X_train_smp.to_csv(io_utils.interim_path + 'x_train_smp.csv')
    y_train_smp.to_csv(io_utils.interim_path + 'y_train_smp.csv')